In [1]:
import math
import numpy as np
import wandb

import torch
import torch_geometric
from torch_geometric.data import Data

from gnn_architectures import MyGnn

import gnn_io as gio
import gnn_architectures as garch

import pprint

## 1. Define model and parameters

In [2]:
wandb.login()

# Define parameters 

num_epochs = 1
batch_size = 20
project_name = 'finetuning model architecture'
train_ratio = 0.8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

early_stopping = gio.EarlyStopping(patience=10, verbose=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enatterer (tum-traffic-engineering). Use `wandb login --relogin` to force relogin


## 2. Load data

In [3]:
# Load the list of dictionaries
data_dict_list = torch.load('../data/dataset_1pm_0-1382.pt')
# torch.load('../data/dataset_1pm_0-1382_with_more_infos.pt') if model_is_basic else 

# Reconstruct the Data objects
datalist = [Data(x=d['x'], edge_index=d['edge_index'], pos=d['pos'], y=d['y']) for d in data_dict_list]

# # Apply normalization to your dataset
dataset_normalized = gio.normalize_dataset(datalist)

baseline_error = gio.compute_baseline_error(datalist)
print(f'Baseline error: {baseline_error}')

Baseline error: 0.005565311759710312


## 3. Split into train and test set

In [4]:
train_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=True, batch_size=20, train_ratio=train_ratio)
valid_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=False, batch_size=20, train_ratio=train_ratio)
# n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

Total dataset length: 1382
Training subset length: 1100
Total dataset length: 1382
Validation subset length: 260


## 4. Train the model

We first find a good model for one batch. 

In [5]:
def train(model, config=None, loss_fct=None, optimizer=None):
    for epoch in range(config.epochs):
        model.train()
        # data = next(iter(train_dl))
        for idx, data in enumerate(train_dl):
            input_node_features, targets = data.x.to(device), data.y.to(device)
            optimizer.zero_grad()

            # Forward pass
            predicted = model(data)
            train_loss = loss_fct(predicted, targets)
            
            # Backward pass
            train_loss.backward()
            optimizer.step()
            
            wandb.log({"train_loss": train_loss.item(), "epoch": epoch, "step": idx})
            # print(f"epoch: {epoch}, step: {idx}, loss: {train_loss.item()}")
        
        val_loss = garch.validate_model_pos_features(model, valid_dl, loss_fct, device)
        print(f"epoch: {epoch}, validation loss: {val_loss}")
        wandb.log({"loss": val_loss, "epoch": epoch})
            
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break
    
    print("Best validation loss: ", val_loss)
    wandb.summary["val_loss"] = val_loss
    wandb.finish()

In [ ]:
batch_size = 16
output_layer_parameter = 'gat'
hidden_size_parameter = 16
gat_layer_parameter = 1
gcn_layer_parameter = 0
lr = 0.001

wandb.login()
counter = 0
# Open the text file for writing
with open('model_performance.txt', 'w') as f:
    train_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=True, batch_size=batch_size, train_ratio=train_ratio)
    valid_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=False, batch_size=batch_size, train_ratio=train_ratio)
    
    wandb.init(
        project=project_name,
        config={
            "epochs": num_epochs,
            "batch_size": batch_size,
            "lr": lr,
            "early_stopping_patience": 10,
            "hidden_layer_size": hidden_size_parameter,
            "gat_layers": gat_layer_parameter,
            "gcn_layers": gcn_layer_parameter,
            "output_layer": output_layer_parameter,
            # "dropout": 0.15,
        }
    )
    config = wandb.config
    
    print("output_layer: ", output_layer_parameter)
    print("hidden_size: ", hidden_size_parameter)
    print("gat_layers: ", gat_layer_parameter)
    print("gcn_layers: ", gcn_layer_parameter)
    
    gnn_instance = MyGnn(in_channels=3, out_channels=1, hidden_size=hidden_size_parameter, gat_layers=gat_layer_parameter, gcn_layers=gcn_layer_parameter, output_layer=output_layer_parameter)
    model = gnn_instance.to(device)
    
    best_val_loss, best_epoch = train(model, config=config, 
                                    loss_fct=torch.nn.MSELoss(), 
                                    optimizer=torch.optim.Adam(model.parameters(), lr=lr),
                                    train_dl=train_dl, valid_dl=valid_dl,
                                    device=device, early_stopping=early_stopping)
    
    # Write the configuration and the best loss to the text file
    f.write(f"Configuration: batch_size={batch_size}, lr={lr}, output_layer={output_layer_parameter}, hidden_size={hidden_size_parameter}, gat_layers={gat_layer_parameter}, gcn_layers={gcn_layer_parameter}\n")
    f.write(f"Best Validation Loss: {best_val_loss}, Best Epoch: {best_epoch}\n\n")